In [17]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import cartopy.crs as ccrs                 
import cartopy
import gsw
import pandas as pd

In [40]:
%matplotlib inline  
#%matplotlib qt5
plt.rcParams['figure.figsize'] = (5,4)
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 400
plt.close('all')

__figdir__ = '../plots/' + '_BoB'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

# define savefig if we want to use if savefig 
savefig = 1

In [27]:
file = '../Data/dataset-duacs-nrt(7:10:219).nc'

In [28]:
ds = xr.open_dataset(file)

In [29]:
ds

<xarray.Dataset>
Dimensions:    (time: 1, latitude: 81, longitude: 80)
Coordinates:
  * time       (time) datetime64[ns] 2019-07-10
  * latitude   (latitude) float32 5.125 5.375 5.625 5.875 ... 24.62 24.88 25.12
  * longitude  (longitude) float32 80.12 80.38 80.62 80.88 ... 99.38 99.62 99.88
Data variables:
    ugos       (time, latitude, longitude) float64 ...
    vgos       (time, latitude, longitude) float64 ...
    sla        (time, latitude, longitude) float64 ...
Attributes: (12/45)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Grid
    comment:                         Sea Surface Height measured by Altimetry...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    creator_email:                   servicedesk.cmems@mercator-ocean.eu
    ...                              ...
    time_coverage_duration:          P1D
    time_coverage_end:               2021-07-13T12:00:00Z
    time_coverage_resolution:        P1D
    time_coverage_start:             2021-07-12T12:00:00Z
    title:                           NRT merged all satellites Global Ocean G...
    History:                         Translated to CF-1.0 Conventions by Netc...

In [30]:
#longitude = [ds.longitude]
#latitude = [ds.latitude]
#lon, lat = np.meshgrid(longitude, latitude)
lon, lat = np.meshgrid(ds.longitude, ds.latitude)
u = np.squeeze(ds.ugos).values
v = np.squeeze(ds.vgos).values

In [31]:
sla = ds.sla

In [ ]:
ax = plt.axes(projection = ccrs.PlateCarree())  # Orthographic

gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.5, linestyle='--')
gl.top_labels = False
gl.ylabels_right = False

levels = 10 # related to dataframe

cs = ax.contourf(ds.longitude,ds.latitude,np.squeeze(sla), levels, extend='both', transform=ccrs.PlateCarree())

'''
cs = (
    ds.sla.squeeze()
    .sel(latitude=slice(15,20))
    .plot.contourf(ax=ax,levels=levels,extend='both', transform=ccrs.PlateCarree())
)
'''

cb = plt.colorbar(cs,ax=ax,shrink=.8,pad=.05)
#cb = plt.colorbar(cs,fraction = 0.022, extend='both')
cb.set_label('SSH (m)',fontsize = 8)
ax.coastlines()
#Plot coast
ax.add_feature(cartopy.feature.LAND, zorder=3, facecolor=[.6,.6,.6], edgecolor='black')

q = ax.quiver(lon,lat, u, v, scale = 25, transform=ccrs.PlateCarree())
qk = ax.quiverkey(q, 0.7, 0.8, 1, r'$1 \frac{m}{s}$',labelpos='N', coordinates='figure')
qk.set(zorder=5)

ax.set_title('Sea Surface Height', fontsize=10, fontweight='bold')

#ax.set_extent([82,90,14,24]) #Set a specific location

if savefig:

    plt.savefig(__figdir__+'_SLA'+plotfiletype,**savefig_args)
    

NameError: name 'plt' is not defined